In [25]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import base64
import cv2
import numpy as np
import PIL
import io
import html
import time
from PIL import Image
import cv2
from scipy import spatial

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
!git clone https://github.com/hoangphu7122002/Yolo-Vehicle-Counter

Cloning into 'Yolo-Vehicle-Counter'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 123 (delta 29), reused 42 (delta 16), pack-reused 60
Receiving objects: 100% (123/123), 39.90 MiB | 19.48 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [4]:
!wget -P /content/ https://pjreddie.com/media/files/yolov3.weights

--2022-04-13 16:21:11--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘/content/yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  72.1MB/s    in 3.5s    

2022-04-13 16:21:15 (67.5 MB/s) - ‘/content/yolov3.weights’ saved [248007048/248007048]



In [8]:
label_path = "/content/Yolo-Vehicle-Counter/yolo-coco/coco.names"

#handle label path
LABELS = open(label_path).read().strip().split("\n")
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")

#handle weight path
weightsPath = "/content/yolov3.weights"


#handle config_path
configPath = "/content/Yolo-Vehicle-Counter/yolo-coco/yolov3.cfg"

YOLOnet = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

In [9]:
ln = YOLOnet.getLayerNames()
ln = [ln[i[0] - 1] for i in YOLOnet.getUnconnectedOutLayers()]

In [10]:
def start_input():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 512, 512);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 512; //video.videoWidth;
      captureCanvas.height = 512; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function takePhoto(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def take_photo(label, img_data):
  data = eval_js('takePhoto("{}", "{}")'.format(label, img_data))
  return data

In [11]:
def js_reply_to_image(js_reply):
    """
    input: 
          js_reply: JavaScript object, contain image from webcam

    output: 
          image_array: image array RGB size 512 x 512 from webcam
    """
    jpeg_bytes = base64.b64decode(js_reply['img'].split(',')[1])
    image_PIL = Image.open(io.BytesIO(jpeg_bytes))
    image_array = np.array(image_PIL)

    return image_array

def bbox_to_bytes(bbox_array):
    """
    Params:
            bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
    Returns:
          bytes: Base64 image byte string
    """
    # convert array into PIL image
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    # format bbox into png for return
    bbox_PIL.save(iobuf, format='png')
    # format return string
    bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

    return bbox_bytes

**Global variable**

In [46]:
FRAMES_BEFORE_CURRENT = 25

**Count object**

In [47]:
def displayVehicleCount(frame, object_count):
    return cv2.putText(
      frame, #Image
      'Detected object: ' + str(object_count), #Label
      (20, 20), #Position
      cv2.FONT_HERSHEY_SIMPLEX, #Font
      0.8, #Size
      (0, 0xFF, 0), #Color
      2, #Thickness
      cv2.FONT_HERSHEY_COMPLEX_SMALL,
      )

def box_in_previous_frame(previous_frame_detections, current_box, current_detections):
    centerX, centerY, width, height = current_box
    dist = np.inf 

    for i in range(FRAMES_BEFORE_CURRENT):
        coordinate_list = list(previous_frame_detections[i].keys())
        
        if len(coordinate_list) == 0: # When there are no detections in the previous frame
          continue
        temp_dist, index = spatial.KDTree(coordinate_list).query([(centerX, centerY)])

        if (temp_dist < dist):
            dist = temp_dist
            frame_num = i
            coord = coordinate_list[index[0]]
        
        if (dist > (max(width, height)/2)):
		        return False
    
    current_detections[(centerX, centerY)] = previous_frame_detections[frame_num][coord]
    return True

def drawDetectionBoxes(idxs, boxes, classIDs, confidences, bbox_array,object_count):
    # ensure at least one detection exists
    if len(idxs) > 0:
      # loop over the indices we are keeping
      for i in idxs.flatten():
        # extract the bounding box coordinates
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])

        # draw a bounding box rectangle and label on the frame
        color = [int(c) for c in COLORS[classIDs[i]]]
        bbox_array = cv2.rectangle(bbox_array, (x, y), (x + w, y + h), color, 2)
        text = "{}: {:.4f}".format(LABELS[classIDs[i]],
          confidences[i])
        bbox_array = cv2.putText(bbox_array, text, (x, y - 5),
          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        #Draw a green dot in the middle of the box
        bbox_array = cv2.circle(bbox_array, (x + (w//2), y+ (h//2)), 2, (0, 0xFF, 0), thickness=2)
        object_count += 1
    return bbox_array, object_count

In [35]:
# def count_object(idxs, boxes, classIDs, object_count, previous_frame_detections, bbox_array,list_of_object = ["person"]):
#     current_detections = {}

#     if len(idxs) > 0:
# 		# loop over the indices we are keeping
#         for i in idxs.flatten():
#           # extract the bounding box coordinates
#           (x, y) = (boxes[i][0], boxes[i][1])
#           (w, h) = (boxes[i][2], boxes[i][3])
          
#           centerX = x + (w//2)
#           centerY = y + (h//2)

#           # When the detection is in the list of vehicles, AND
#           # it crosses the line AND
#           # the ID of the detection is not present in the vehicles
#           if (LABELS[classIDs[i]] in list_of_object):
#               current_detections[(centerX, centerY)] = object_count 
#               if (not box_in_previous_frame(previous_frame_detections, (centerX, centerY, w, h), current_detections)):
#                   object_count += 1
#                 # vehicle_crossed_line_flag += True
#               # else: #ID assigning
#                 #Add the current detection mid-point of box to the list of detected items
#               # Get the ID corresponding to the current detection

#               ID = current_detections.get((centerX, centerY))
#               # If there are two detections having the same ID due to being too close, 
#               # then assign a new ID to current detection.
#               if (list(current_detections.values()).count(ID) > 1):
#                 current_detections[(centerX, centerY)] = object_count
#                 object_count += 1 

#               #Display the ID at the center of the box
#               bbox_array = cv2.putText(bbox_array, str(ID), (centerX, centerY),cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0,0,255], 2)
    
#     return object_count, current_detections

In [50]:
previous_frame_detections = [{(0,0):0} for i in range(FRAMES_BEFORE_CURRENT)]
num_frames, object_count = 0, 0

start_input()

label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
while True:
    js_reply = take_photo(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_reply_to_image(js_reply)

    W,H = img.shape[:2]

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # # grayscale image for face detection
    # gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # # get face region coordinates
    # faces = face_cascade.detectMultiScale(gray)
    # # get face bounding box for overlay
    # for (x,y,w,h) in faces:
    #   bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)

    blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416),
		swapRB=True, crop=False)

    YOLOnet.setInput(blob)
    start = time.time()
    layerOutputs = YOLOnet.forward(ln)
    end = time.time()
    
    boxes = []
    confidences = []
    classIDs = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            if confidence > 0.9:
                box = detection[0:4] * np.array([W, H, W, H])
                centerX, centerY, width, height = box.astype("int")

                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.9, 0.3)

    # if len(idxs) > 0:
    #   for i in idxs.flatten():
    #     (x, y) = (boxes[i][0], boxes[i][1])
    #     (w, h) = (boxes[i][2], boxes[i][3])
    #     color = [int(c) for c in COLORS[classIDs[i]]]
    #     bbox_array = cv2.rectangle(bbox_array, (x, y), (x + w, y + h), color, 2)
    #     text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
    #     bbox_array = cv2.putText(bbox_array, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
    #       0.5, color, 2) 
    bbox_array,object_count = drawDetectionBoxes(idxs, boxes, classIDs, confidences, bbox_array,object_count)
    # object_count,current_detections = count_object(idxs, boxes, classIDs, object_count, previous_frame_detections, bbox_array,list_of_object = ["person"])
    displayVehicleCount(bbox_array, object_count)
    object_count = 0
    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

    if cv2.waitKey(1) & 0xFF == ord('q'):
		   break	

    # previous_frame_detections.pop(0)
    # previous_frame_detections.append(current_detections)



<IPython.core.display.Javascript object>